In [1]:
using Revise

In [2]:
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using MeshCat
using MeshCatMechanisms
using AtlasRobot
using Blink
using RigidBodySim
using LCPSim
using LearningMPC
using StaticArrays
using Gurobi

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LCPSim.ji for module LCPSim.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [3]:
vis = Visualizer()
open(vis, Window())
# manip_window = Window()

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 4597`, ProcessRunning), TCPSocket(RawFD(53) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#1))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(56) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#1)), Future(1, 1, 1, Nullable{Any}(true))))

In [9]:
delete!(vis)
urdf_mech = parse_urdf(Float64, "atlas.urdf")

mechanism, base = LCPSim.xyz_rpy_floating_base()
submech, _, _ = submechanism(urdf_mech, findbody(urdf_mech, "pelvis"))
attach!(mechanism, base, submech)

# mechanism.gravitational_acceleration = FreeVector3D(root_frame(mechanism), 0.,0, 0)

# pelvis = findbody(mechanism, "pelvis")
# basejoint = joint_to_parent(pelvis, mechanism)
# floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint),
#     QuaternionFloating{Float64}())
# effort_bounds(floatingjoint) .= RigidBodyDynamics.Bounds(-0, 0)
# position_bounds(floatingjoint) .= RigidBodyDynamics.Bounds(-10, 10)
# velocity_bounds(floatingjoint) .= RigidBodyDynamics.Bounds(-1000, 1000)
# replace_joint!(mechanism, basejoint, floatingjoint)
# basejoint = floatingjoint

contactmodel = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(1.0, 20e3, 100.))
for (name, offset) in [
        ("l_lleg", [0, 0, -0.452]),
        ("r_lleg", [0, 0, -0.452]),
        ("l_ufarm", [0, -0.05, 0]),
        ("r_ufarm", [0, -0.05, 0])
    ]
    body = findbody(mechanism, name)
    frame = default_frame(body)
    add_contact_point!(body, ContactPoint(Point3D(frame, offset...), contactmodel))
end

add_environment_primitive!(mechanism, HalfSpace3D(Point3D(root_frame(mechanism), 0., 0, 0),
        FreeVector3D(root_frame(mechanism), 0., 0, 1)))

mvis = MechanismVisualizer(mechanism, URDFVisuals("atlas.urdf", package_path=[AtlasRobot.packagepath()]), vis)
# body!(manip_window, manipulate!(mvis))

MeshCatMechanisms.MechanismVisualizer{RigidBodyDynamics.MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Fixed{Float64}},1},Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Prismatic{Float64}},1},Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Revolute{Float64}},1}},3}},MeshCat.Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat, 28)

In [10]:
function init!(state::MechanismState)
    m = state.mechanism
    kneebend = 1.0
    hipbendextra = -0.0625
    set_configuration!(state, findjoint(m, "base_z"), 0.82)
    for sideprefix in ('l', 'r')
        knee = findjoint(m, "$(sideprefix)_leg_kny")
        hippitch = findjoint(m, "$(sideprefix)_leg_hpy")
        shoulderroll = findjoint(m, "$(sideprefix)_arm_shx")
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        if sideprefix == 'r'
            set_configuration!(state, shoulderroll, 1)
        else
            set_configuration!(state, shoulderroll, -1)
        end
    end
    state
end

init! (generic function with 1 method)

In [11]:
copy!(mvis, init!(MechanismState(mechanism)))

In [12]:
state = init!(MechanismState(mechanism))
transform(state, location(first(contact_points(findbody(mechanism, "r_lleg")))), root_frame(mechanism))

Point3D in "world": [0.0156573, -0.1115, 0.00153132]

In [13]:
center_of_mass(state)

Point3D in "world": [0.0160549, 0.00116997, 1.13693]

In [14]:
state = init!(MechanismState(mechanism))
τc = similar(velocity(state))
τc .= 0
RigidBodyDynamics.constraint_bias!(τc, state)
u0 = inverse_dynamics(state, τc)

16-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
    0.0     
 1540.24    
    1.80204 
    0.0     
    2.95759 
   -2.95759 
  -12.5053  
  -12.5053  
    0.0     
    0.0     
    3.46903 
    3.46903 
   13.6341  
  -13.6341  
   -0.196106
    0.180958

In [24]:
qq = fill(0.1, num_positions(mechanism) + num_velocities(mechanism))
qq[1:num_positions(mechanism)] .= 1
qq[1:2] .= [100, 10]
Q = diagm(qq)
R = diagm(fill(1e-5, num_velocities(mechanism)))
Δt = 0.001
contacts = [
    Point3D(default_frame(findbody(mechanism, name)), offset...)
    for (name, offset) in [
        ("l_lleg", [0, 0, -0.452]),
        ("r_lleg", [0, 0, -0.452])
    ]
]
# contacts = Point3D[]
lqrsol = LQRSolution(init!(MechanismState(mechanism)),
    Q, R, Δt, contacts
)

(::LQRSolution) (generic function with 2 methods)

A = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

In [25]:
floor_obstacle = LCPSim.planar_obstacle(root_frame(mechanism), SVector(0., 0, 1), SVector(0., 0, 0))

env = LCPSim.Environment([
    (body, Point3D(default_frame(body), 0, 0, -0.452), floor_obstacle)
    for body in findbody.(mechanism, ["l_lleg", "r_lleg"])
]);

In [44]:
qnext = configuration(init!(MechanismState(mechanism)))

16-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
  0.0   
  0.82  
  0.0   
  0.0   
  0.0   
  0.0   
 -0.5625
 -0.5625
  0.0   
  0.0   
  1.0   
  1.0   
 -1.0   
  1.0   
  0.0   
  0.0   

In [46]:
qnext[2] + 0.11149999999999999qnext[3] + 0.0225qnext[5] + 0.03434269331526185qnext[7] + 0.19150166825617998qnext[11] - 0.9906525845640889

0.0015313187022562014

In [51]:
state = init!(MechanismState(mechanism))
set_velocity!(state, 0.01 .* randn(num_velocities(state)))
mpc_controller = LearningMPC.MPCController(
    MechanismState(mechanism), 
    env,
    MPCParams(
        Δt=Δt,
        horizon=1,
        mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, TimeLimit=10),
        lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, TimeLimit=10)
    ),
    lqrsol,
    Function[],
    (state, results) -> nothing
)
controller = let lqr = lqrsol, bounds = LCPSim.all_effort_bounds(mechanism), mpc = mpc_controller
    function(τ, t, x)
        τ .= mpc(x)
#         τ .= lqr(x)
#         τ .= clamp.(τ, bounds)
#         τ .= 0
#         @show τ
    end
end

problem = LearningMPC.simulation_problem(state, controller, Δt, 0.01)


# periodic = PeriodicController(similar(velocity(state)), Δt, controller)
# dynamics = Dynamics(mechanism, periodic)


# problem = ODEProblem(dynamics, state, (0., 5.0))
sol = solve(problem, Tsit5(), abs_tol=1e-8, dt=1e-6)
setanimation!(mvis, sol)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
separation_from_obstacle = qnext[2] + 0.11149999999999999 qnext[3] + 0.0225 qnext[5] + 0.03434269331526185 qnext[7] + 0.19150166825617998 qnext[11] - 0.9906525845640889
separation_from_obstacle = qnext[2] - 0.11149999999999999 qnext[3] - 0.0225 qnext[6] + 0.03434269331526185 qnext[8] + 0.19150166825617998 qnext[12] - 0.9906525845640889
separation_from_obstacle = qnext[2] + 0.11150140277001121 qnext[3] - 2.9483184194657362e-8 qnext[4] + 0.022501374006460923 qnext[5] + 0.03434280387648961 qnext[7] + 0.19150240358493212 qnext[11] - 0.9906532577044789
separation_from_obstacle = qnext[2] - 0.11149210466831602 qnext[3] - 2.9518646802041496e-8 qnext[4] - 0.02249213346640261 qnext[6] + 0.03432438118411646 qnext[8] + 0.19149385443698022 qnext[12] - 0.9906550715916144
separation_from_obstacle = qnext[2] + 0.11150267141259647 qnext[3] - 5.406972977170962e-8 qnext[4] + 0.022502617192758062 qnext[5] + 0.03

true

separation_from_obstacle = qnext[2] + 0.1115098131084198 qnext[3] - 2.3186799199328662e-7 qnext[4] + 0.022509503618698235 qnext[5] + 0.034323051625761186 qnext[7] + 0.19148067823595924 qnext[11] - 0.9906426435854994
separation_from_obstacle = qnext[2] - 0.11143994251330011 qnext[3] - 2.3334057941023995e-7 qnext[4] - 0.022440253944526607 qnext[6] + 0.03422790349584223 qnext[8] + 0.1914520732521289 qnext[12] - 0.990667570208556


In [52]:
sol.u

19-element Array{Array{Float64,1},1}:
 [0.0, 0.82, 0.0, 0.0, 0.0, 0.0, -0.5625, -0.5625, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                                                           
 [1.83558e-8, 0.82, -1.4457e-9, 1.49312e-9, 2.37754e-9, 9.22916e-9, -0.5625, -0.5625, 7.50779e-9, 6.51722e-10  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]         
 [2.01905e-7, 0.82, -1.59197e-8, 1.6433e-8, 2.62197e-8, 1.01586e-7, -0.5625, -0.5625, 8.25567e-8, 7.16257e-9  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]          
 [2.03647e-6, 0.82, -1.62364e-7, 1.66692e-7, 2.71319e-7, 1.03164e-6, -0.5625, -0.562503, 8.30141e-7, 7.16325e-8  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]       
 [1.8272e-5, 0.819999, -1.60046e-6, 1.57145e-6, 2.98378e-6, 9.81861e-6, -0.562502, -0.562526, 7.24388e-6, 5.93237e-7  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
 [3.64766e-5, 0.819987, -3.32898e-6, 3.32349e-6, 6.41884e-6, 2.00554e-5, -0.562502, -0.562551, 1.

In [53]:
sol.t

19-element Array{Float64,1}:
 0.0       
 1.0e-6    
 1.1e-5    
 0.000111  
 0.001     
 0.002     
 0.00246515
 0.003     
 0.004     
 0.00472161
 0.005     
 0.006     
 0.00683987
 0.007     
 0.008     
 0.00890598
 0.009     
 0.00994016
 0.01      